# Social Network Analysis - Breaking Hits
## Authors: Wing Yan Sang and Iman Singh
## Date: 12/12/2017

<p><a name="sections"></a></p>


## Sections
- <a href="#pre-processing"> Pre-Processing</a><br>
- <a href="#graphs"> Network Graphs</a><br>
- <a href="#centrality"> Centrality</a><br>
 - <a href="#centrality1">Centrality Analysis 1: "Celebrities"</a><br>
 - <a href="#centrality2">Centrality Analysis 2: "Gossipmongers"</a><br>
 - <a href="#centrality3">Centrality Analysis 3: "Boundary Spanners"</a><br>
 - <a href="#centrality4">Centrality Analysis 4: "Gray Cardinals"</a><br>
 - <a href="#centrality5">Centrality Analysis 5: "Page Rank"</a><br>   
 - <a href="#centrality_summary">Centrality Analysis: Summary </a><br>   
- <a href="#ccc"> Cliques, Clusters, Components</a><br>

In [1]:
#import mysql.connector
import pandas as pd
import numpy as np
import mysql.connector as sql
import breakinghits_script as bh

from matplotlib import pyplot as plt
import networkx as nx
from networkx.algorithms import bipartite
%matplotlib inline

pd.options.display.max_rows = 1000

<p><a name="pre-processing"></a></p>
## Pre-Processing

In [2]:
# This function pulls specified data from Breaking Hits MySQL server
# parameters: 'table_name' as string, 'col_names' as list of strings

def get_table(table_name, col_names):    
    cnx = sql.connect(user='breaking_read',
                                  password='(hHy;gTPMnet',
                                  host='206.225.82.147',
                                  database='breaking_livedb')
    cur = cnx.cursor()
    select_statement = 'SELECT ' + ', '.join(col_names) + ' FROM ' + table_name +';'
    cur.execute(select_statement)
    rows = cur.fetchall()                         # get all selected rows
    field_names = [i[0] for i in cur.description] # get all column names
    cnx.close() # close the sql connection

    table = pd.DataFrame(rows, columns=field_names)
    return table

In [3]:
#removes rows with missing data 

def dropMissing(df):
    df = df.replace('', np.nan, regex=True) #convert empty cells to nan
    df = df.dropna()
    return df


In [4]:
#converts object to int64 for first two cols in data frame. (assumes no missing values)

def convertInt(df):
    for col in df.columns[:1]:
        if (df[col].dtype == 'object'):
            df[col] = df[col].astype(int)
    return df


In [5]:
# Parameters string specifying connection type: follow, message, view, sociallink, rating, save, spotlight, comment
# Returns formatted pandas dataframe

def get_dataframe(connection='follow'):  
    if (connection == 'follow'):
        df = get_table('user_following', ['followed_id', 'follower_id', 'date_followed'])
    elif (connection == 'sociallink'):
        df = get_table('bh_social_link_activity', ['user_id_shared', 'user', 'date_shared'])
    elif (connection == 'message'):
        df = get_table('bh_messenger', ['sendfrom', 'sendto', 'u_read', 'data_added'])
    elif (connection == 'view'):
        df = get_table('bh_music_views', ['artists_id', 'user_id', 'date_added'])
    elif (connection == 'rating'):
        df = get_table('user_music_votes', ['user_id', 'user_music_id', 'rating', 'date_added'])
    elif (connection == 'save'):
        df = get_table('user_saved', ['music_id', 'user_id', 'date_saved'])   
    elif (connection == 'spotlight'):
        df = get_table('user_spotlight', ['music_id', 'user_id', 'date_added'])   
    elif (connection == 'comment'):
        df = get_table('user_music_comments', ['user_music_id', 'user_id', 'date_added'])
    else:
        return
   
    #Drop missing rows and convert and convert object to int for each column:   
    df = dropMissing(df)
    df = convertInt(df)
    
    return df

In [6]:
#Draw edges between nodes of a graph or directed graph
#Parameters: graph object (directed or undirected), dataframe that contains network info
#Returns: graph object with edges drawn

def add_edges(G, df):
    for row in df.itertuples():
        if ('u_read' in list(df)):
            G.add_edge(row[1], row[2], read=row[3], date=row[4])
        elif ('rating' in list(df)):
            G.add_edge(row[1], row[2], rating=row[3], date=row[4])
        else:
            G.add_edge(row[2], row[1], date=row[3])
    return G

#Define function to filter graph object to only include those that are tagged as critic/artist and label
#nodes as critic/artist
def get_artcric_list(G):
    listeners, artists = bh.users()
    l = listeners['id'].tolist()
    a = artists['id'].tolist()
    l_list = [x for x in l if x in set(G.nodes())] #list of critics 
    a_list = [x for x in a if x in set(G.nodes())] #list of artists
    s1 = l_list + a_list #combined list of critics and artists 
    s2 = list(set(s1) & set(G.nodes())) #list of people in G.nodes() and tagged as artist or critic
    
    return s2

#define node attribute 'user_type' = artist or critic
def add_artcric_keys(G):
    key_list = G.nodes()
    listeners, artists = bh.users()
    l = listeners['id'].tolist()
    a = artists['id'].tolist()
    l_list = [x for x in l if x in set(G.nodes())] #list of critics 
    a_list = [x for x in a if x in set(G.nodes())] #list of artists
    
    for i in range(0,len(key_list)): 
        if (key_list[i] in l_list):
            G.node[key_list[i]]['user_type'] = 'critic'
        else:
            G.node[key_list[i]]['user_type'] = 'artist'
    return G

In [7]:
#define node attribute 'user_type' = artist or critic
def filter_graph_artcric(G):
    key_list = G.nodes()
    listeners, artists = bh.users()
    l = listeners['id'].tolist()
    a = artists['id'].tolist()
    l_list = [x for x in l if x in set(G.nodes())] #list of critics 
    a_list = [x for x in a if x in set(G.nodes())] #list of artists
    s1 = l_list + a_list #combined list of critics and artists 
    s2 = list(set(s1) & set(G.nodes())) #list of people in G.nodes() and tagged as artist or critic
    
    G_reduced = nx.Graph(G.subgraph(s2))
    
    for i in range(0,len(key_list)): 
        if (key_list[i] in l_list):
            G_reduced.node[key_list[i]]['user_type'] = 'critic'
        else:
            G_reduced.node[key_list[i]]['user_type'] = 'artist'
    return G_reduced

In [8]:
#define node attribute 'user_type' = artist or critic
def filter_digraph_artcric(DG):
    key_list = DG.nodes()
    listeners, artists = bh.users()
    l = listeners['id'].tolist()
    a = artists['id'].tolist()
    l_list = [x for x in l if x in set(DG.nodes())] #list of critics 
    a_list = [x for x in a if x in set(DG.nodes())] #list of artists
    s1 = l_list + a_list #combined list of critics and artists 
    s2 = list(set(s1) & set(DG.nodes())) #list of people in G.nodes() and tagged as artist or critic
    
    DG_reduced = nx.DiGraph(DG.subgraph(s2))
    
    for i in range(0,len(key_list)): 
        if (key_list[i] in l_list):
            DG_reduced.node[key_list[i]]['user_type'] = 'critic'
        else:
            DG_reduced.node[key_list[i]]['user_type'] = 'artist'
    return DG_reduced

#Define function to filter graph object to only include those that are tagged as critic/artist and label
#nodes as critic/artist. No direction to the edges.
def filter_graph(G):
    G_reduced = nx.Graph(G.subgraph(get_artcric_list(G)))
    G_reduced = add_artcric_keys(G_reduced)
    return G_reduced

In [ ]:
def get_userlists(df):
    userlist = sorted(set(df['user_id'].tolist()))
    return userlist

def get_musiclists_1(df):
    musiclist = sorted(set(df['user_music_id'].tolist()))
    return musiclist

def get_musiclists_2(df):
    musiclist = sorted(set(df['music_id'].tolist()))
    return musiclist

In [9]:
#Parameter: dataframe that user wants to use for creating bipartite graph
#mutates dataframe into one that has negated value for music_id

def negate_musicid_1(df):
    df['user_music_id'] = -1 * df_rating['user_music_id']   

def negate_musicid_2(df):
    df['music_id'] = -1 * df['music_id']

In [10]:
#Parameter: dataframe to create bipartite graph
#Returns: bipartite graph

def get_bigraph_1(df):
    BG = nx.Graph()
    BG.add_nodes_from(df['user_id'], bipartite = 0)
    BG.add_nodes_from(df['user_music_id'], bipartite=1)
    zipped_edges = zip(df['user_id'], df['user_music_id'])
    BG.add_edges_from(zipped_edges)
    return BG

def get_bigraph_2(df):
    BG = nx.Graph()
    BG.add_nodes_from(df['user_id'], bipartite = 0)
    BG.add_nodes_from(df['music_id'], bipartite=1)
    zipped_edges = zip(df['user_id'], df['music_id'])
    BG.add_edges_from(zipped_edges)
    return BG

In [11]:
#Parameter: bipartite graph opbject
#Output: 2 lists: top_nodes and bottom_nodes

def get_bipartite_nodes(BG):
    top_nodes, bottom_nodes = bipartite.sets(B)
    
    top_nodes = {n for n, d in B.nodes(data=True) if d['bipartite']==0}
    bottom_nodes = set(B) - top_nodes
    
    return top_nodes, bottom_nodes

In [12]:
#Parameter: dataframe used for bipartite graph
#Output: 'pos' argument for use for graph axes

def get_pos_1(df):
    pos = {node:[0, i] for i,node in enumerate(df['user_id'])}
    pos.update({node:[1, i] for i,node in enumerate(df['user_music_id'])})
    return pos

def get_pos_2(df):
    pos = {node:[0, i] for i,node in enumerate(df['user_id'])}
    pos.update({node:[1, i] for i,node in enumerate(df['music_id'])})
    return pos

In [13]:
#examine missingness of the various dataframes:
connection = ['follow', 'message', 'view', 'sociallink', 'rating', 'save', 'spotlight', 'comment']
[print(bh.countMissing(get_dataframe(i)),'\n') for i in connection]


               Count  Percentage
followed_id        0         0.0
follower_id        0         0.0
date_followed      0         0.0 

            Count  Percentage
sendfrom        0         0.0
sendto          0         0.0
u_read          0         0.0
data_added      0         0.0 

            Count  Percentage
artists_id      0         0.0
user_id         0         0.0
date_added      0         0.0 

                Count  Percentage
user_id_shared      0         0.0
user                0         0.0
date_shared         0         0.0 

               Count  Percentage
user_id            0         0.0
user_music_id      0         0.0
rating             0         0.0
date_added         0         0.0 

            Count  Percentage
music_id        0         0.0
user_id         0         0.0
date_saved      0         0.0 

            Count  Percentage
music_id        0         0.0
user_id         0         0.0
date_added      0         0.0 

               Count  Percentage
user_music

[None, None, None, None, None, None, None, None]

In [14]:
#examine column types for each data frame

connection = ['follow', 'message', 'view', 'sociallink', 'rating', 'save', 'spotlight', 'comment']

for i in connection:
    print(i, " dataframe", '\n')
    print(get_dataframe().dtypes)

follow  dataframe 

followed_id               int64
follower_id               int64
date_followed    datetime64[ns]
dtype: object
message  dataframe 

followed_id               int64
follower_id               int64
date_followed    datetime64[ns]
dtype: object
view  dataframe 

followed_id               int64
follower_id               int64
date_followed    datetime64[ns]
dtype: object
sociallink  dataframe 

followed_id               int64
follower_id               int64
date_followed    datetime64[ns]
dtype: object
rating  dataframe 

followed_id               int64
follower_id               int64
date_followed    datetime64[ns]
dtype: object
save  dataframe 

followed_id               int64
follower_id               int64
date_followed    datetime64[ns]
dtype: object
spotlight  dataframe 

followed_id               int64
follower_id               int64
date_followed    datetime64[ns]
dtype: object
comment  dataframe 

followed_id               int64
follower_id               int64
d

<p><a name="graphs"></a></p>
## Network Graphs

-- View Follow/Followed network 

-- Define function to filter graph to only include users tagged critics/artists and to add attribute artist/critic

In [16]:
#call function to get graph and dataframes for all four connection types
df_follow = get_dataframe('follow')
df_message = get_dataframe('message')
df_view = get_dataframe('view')
df_sociallink = get_dataframe('sociallink')

In [17]:
G_follow = add_edges(nx.Graph(), df_follow)
DG_follow = add_edges(nx.DiGraph(), df_follow)
G_message = add_edges(nx.Graph(), df_message)
DG_message = add_edges(nx.DiGraph(), df_message)
G_view = add_edges(nx.Graph(), df_view)
DG_view = add_edges(nx.DiGraph(), df_view)
G_sociallink = add_edges(nx.Graph(), df_sociallink)
DG_sociallink = add_edges(nx.DiGraph(), df_sociallink)

In [18]:
G_follow = filter_graph_artcric(G_follow)
DG_follow = filter_digraph_artcric(DG_follow)
G_message = filter_graph_artcric(G_message)
DG_message = filter_digraph_artcric(DG_message)
G_view = filter_graph_artcric(G_view)
DG_view = filter_digraph_artcric(DG_view)
G_sociallink = filter_graph_artcric(G_sociallink)
DG_sociallink = filter_digraph_artcric(DG_sociallink)

KeyError: 373

In [ ]:
#plot of network, whether tagged or not as listener/artist
plt.figure(figsize=(20,15))
plt.title('Network Graph of Follower/Followed', fontsize = 20)
nx.draw_networkx(GF, node_size = 500, alpha = 0.4)

In [ ]:
#plot of network, whether tagged or not as listener/artist
plt.figure(figsize=(20,15))
nx.draw_networkx(DGF, node_size = 500, alpha = 0.3)

In [ ]:
#Example using follow/followed connection
G_reduced = nx.Graph(GF.subgraph(get_artcric_list(GF)))
DG_reduced = nx.Graph(DGF.subgraph(get_artcric_list(DGF)))

In [ ]:
len(get_artcric_list(GF))

In [ ]:
#Define function to filter graph object to only include those that are tagged as critic/artist and label
#nodes as critic/artist. Edges marked with direction.
def filter_digraph(DG):
    DG_reduced = nx.DiGraph(DG.subgraph(get_artcric_list(DG)))    
    DG_reduced = add_artcric_keys(DG)
    return DG_reduced

In [ ]:
#implement filter function and draw the filtered graph
GF_reduced = filter_graph(GF)
color_map = {}
key_list = GF_reduced.nodes()

for i in range(0, len(key_list)):
    if(GF_reduced.node[key_list[i]]['user_type'] == 'critic'):
        color_map[key_list[i]] = 'b'
    else:
        color_map[key_list[i]] = 'r'

colors = [color_map.get(node) for node in GF_reduced.nodes()]
plt.figure(figsize=(20,20))
nx.draw_networkx(GF_reduced, node_color = colors, node_size = 500, alpha = 0.4)

In [ ]:
len(GF_reduced.nodes())

In [ ]:
DGF_reduced = filter_digraph(DGF)
key_list = DGF_reduced.nodes()

for i in range(0, len(key_list)):
    if(DGF_reduced.node[key_list[i]]['user_type'] == 'critic'):
        color_map[key_list[i]] = 'b'
    else:
        color_map[key_list[i]] = 'r'
        
colors = [color_map.get(node) for node in DGF_reduced.nodes()]
plt.figure(figsize=(20,20))
nx.draw_networkx(DGF_reduced, node_color = colors, node_size = 500, alpha = 0.4)

In [ ]:
#implement filter function and draw the filtered graph
GM_reduced = filter_graph(GM)
color_map = {}
key_list = GM_reduced.nodes()

for i in range(0, len(key_list)):
    if(GM_reduced.node[key_list[i]]['user_type'] == 'critic'):
        color_map[key_list[i]] = 'b'
    else:
        color_map[key_list[i]] = 'r'

colors = [color_map.get(node) for node in GM_reduced.nodes()]
plt.figure(figsize=(20,15))
nx.draw_networkx(GM_reduced, node_color = colors, node_size = 500, alpha = 0.4)

In [ ]:
DGM_reduced = filter_digraph(DGM)
color_map = {}
key_list = DGM_reduced.nodes()
for i in range(0, len(key_list)):
    if(DGM_reduced.node[key_list[i]]['user_type'] == 'critic'):
        color_map[key_list[i]] = 'b'
    else:
        color_map[key_list[i]] = 'r'
    
colors = [color_map.get(node) for node in DGM_reduced.nodes()]
plt.figure(figsize=(20,20))
nx.draw_networkx(DGM_reduced, node_color = colors, node_size = 500, alpha = 0.4)

In [ ]:
#implement filter function and draw the filtered graph
GV_reduced = filter_graph(GV)
color_map = {}
key_list = GV_reduced.nodes()

for i in range(0, len(key_list)):
    if(GV_reduced.node[key_list[i]]['user_type'] == 'critic'):
        color_map[key_list[i]] = 'b'
    else:
        color_map[key_list[i]] = 'r'

colors = [color_map.get(node) for node in GV_reduced.nodes()]
plt.figure(figsize=(20,15))
nx.draw_networkx(GV_reduced, node_color = colors, node_size = 500, alpha = 0.4)

In [ ]:
DGV_reduced = filter_digraph(DGV)
color_map = {}
key_list = DGV_reduced.nodes()

for i in range(0, len(key_list)):
    if(DGV_reduced.node[key_list[i]]['user_type'] == 'critic'):
        color_map[key_list[i]] = 'b'
    else:
        color_map[key_list[i]] = 'r'
        
colors = [color_map.get(node) for node in DGV_reduced.nodes()]
plt.figure(figsize=(20,20))
nx.draw_networkx(DGV_reduced, node_color = colors, node_size = 500, alpha = 0.4)

In [ ]:
#implement filter function and draw the filtered graph
GS_reduced = filter_graph(GS)
color_map = {}
key_list = GS_reduced.nodes()

for i in range(0, len(key_list)):
    if(GS_reduced.node[key_list[i]]['user_type'] == 'critic'):
        color_map[key_list[i]] = 'b'
    else:
        color_map[key_list[i]] = 'r'

colors = [color_map.get(node) for node in GS_reduced.nodes()]
plt.figure(figsize=(20,15))
nx.draw_networkx(GS_reduced, node_color = colors, node_size = 500, alpha = 0.4)

In [ ]:
DGS_reduced = filter_digraph(DGS)
key_list = DGS_reduced.nodes()
color_map = {}

for i in range(0, len(key_list)):
    if(DGS_reduced.node[key_list[i]]['user_type'] == 'critic'):
        color_map[key_list[i]] = 'b'
    else:
        color_map[key_list[i]] = 'r'
        
colors = [color_map.get(node) for node in DGS_reduced.nodes()]
plt.figure(figsize=(20,15))
nx.draw_networkx(DGS_reduced, node_color = colors, node_size = 500, alpha = 0.4)

<p><a name="centrality1"></a></p>
## Centrality Analysis #1: "Celebrities"

In [ ]:
import operator

#defines a function that returns a sorted list of people (1st element) and the number of connections (2nd element)
def sorted_map(map):
    sorted_map = sorted(map.items(), key=operator.itemgetter(1), reverse = True) 
    return(sorted_map)


In [ ]:
#Define a function that returns top n "celebrities" in form of data frame

def getCeleb(g, n, user_type =None):
    d=nx.degree(g) 
    ds=sorted_map(d)
    ds_critic = []
    ds_artist = []
    
    listeners, artists = bh.users()
    l = listeners['id'].tolist()
    a = artists['id'].tolist()
    l_list = [x for x in l if x in set(g.nodes())] #list of critics 
    a_list = [x for x in a if x in set(g.nodes())] 
    
    for i in range(0,len(ds)):
        if (ds[i][0] in l_list):
            ds_critic.append(ds[i])
        else:
            ds_artist.append(ds[i])
    
    if (user_type == 'critic'):
        temp = ds_critic
    elif (user_type == 'artist'):
        temp = ds_artist[0:n]
    else:
        temp = ds[0:n]
    
    import heapq
    b = [x[1] for x in temp]
    b_n = min(heapq.nlargest(n, set(b)))
    ans = [x for x in temp if x[1]>=b_n]  
    
    return(pd.DataFrame(ans, columns = ["id", "num_connections"]))




In [ ]:
#Test function
g = get_graph(get_dataframe('follow')) 
g_reduced = filter_graph(g)
n = 5
user_type = 'critic'
print("Top %d %ss by number of connections (%s, #connections):" %(n,user_type, user_type))
getCeleb(g_reduced, n, user_type)

In [ ]:
#Test function
n = 5
user_type = 'artist'
print("Top %d %ss by number of connections (%s, #connections):" %(n,user_type, user_type))
getCeleb(g_reduced, n, user_type)

In [ ]:
#Test function
n = 5
user_type = 'all user'
print("Top %d %ss by number of connections (%s, #connections):" %(n,user_type, user_type))
getCeleb(g_reduced, n, user_type)

<p><a name="centrality2"></a></p>
## Centrality Analysis #2: "Gossipmongers"

In [ ]:
#Function that returns top n "gossipmongers" by critic/artist in form of data frame

def getClose(g, n, user_type =None):
    c = nx.closeness_centrality(g)
    cs=sorted_map(c)
    cs_critic = []
    cs_artist = []
    
    listeners, artists = bh.users()
    l = listeners['id'].tolist()
    a = artists['id'].tolist()
    l_list = [x for x in l if x in set(g.nodes())] #list of critics 
    a_list = [x for x in a if x in set(g.nodes())] 
    
    for i in range(0,len(cs)):
        if (cs[i][0] in l_list):
            cs_critic.append(cs[i])
        else:
            cs_artist.append(cs[i])
    
    if (user_type == 'critic'):
        ans = cs_critic[0:n]
    elif (user_type == 'artist'):
        ans = cs_artist[0:n]
    else:
        ans =cs[0:n]
    return(pd.DataFrame(ans, columns = ["id", "closeness"]))


In [ ]:
#Test function
n = 5
user_type = 'critic'
print("Top %d %ss by closeness measure (%s, closeness):" %(n,user_type, user_type))
getClose(g_reduced, n, user_type)

In [ ]:
#Test function
n = 5
user_type = 'artist'
print("Top %d %ss by closeness measure (%s, closeness):" %(n,user_type, user_type))
getClose(g_reduced, n, user_type)

In [ ]:
#Test function
n = 5
user_type = 'all user'
print("Top %d %ss by closeness measure (%s, closeness):" %(n,user_type, user_type))
getClose(g_reduced, n, user_type)

<p><a name="centrality3"></a></p>
## Centrality Analysis #3: "Boundary Spanners"

In [ ]:
#Function that returns top n "boundary spanners" by critic/artist

def getSpan(g, n, user_type=None):
    b = nx.betweenness_centrality(g)
    bs=sorted_map(b)
    bs_critic = []
    bs_artist = []
    
    listeners, artists = bh.users()
    l = listeners['id'].tolist()
    a = artists['id'].tolist()
    l_list = [x for x in l if x in set(g.nodes())] #list of critics 
    a_list = [x for x in a if x in set(g.nodes())]
    
    for i in range(0,len(bs)):
        if (bs[i][0] in l_list):
            bs_critic.append(bs[i])
        else:
            bs_artist.append(bs[i])
    
    if (user_type == 'critic'):
        ans = bs_critic[0:n]
    elif (user_type == 'artist'):
        ans = bs_artist[0:n]
    else:
        ans = bs[0:n]
    return(pd.DataFrame(ans, columns = ["id", "betwenness"]))


In [ ]:
#Test function
n = 5
user_type = "critic"
print("Top %d %ss by betweenness measure (%s_id, betweenness):" %(n,user_type, user_type))
getSpan(g_reduced, n, user_type)

In [ ]:
#Test function
n = 5
user_type = "artist"
print("Top %d %ss by betweenness measure (%s_id, betweenness):" %(n,user_type, user_type))
getSpan(g_reduced, n, user_type)

In [ ]:
#Test function
n = 5
user_type = "all user"
print("Top %d %ss by betweenness measure (%s_id, betweenness):" %(n,user_type, user_type))
getSpan(g_reduced, n, user_type)

<p><a name="centrality4"></a></p>
## Centrality Analysis #4: "Gray Cardinals"

In [ ]:
#calculate eigenvalues
e = nx.eigenvector_centrality_numpy(g_reduced)
es = sorted_map(e)
es

#check the eigenvalues are greater than 0 
#(https://stackoverflow.com/questions/43208737/using-networkx-to-calculate-eigenvector-centrality)
es[-1][1]>0

# define function that returns top n people by eigenvalue
def getEigen(g, n, user_type =None):
    e = nx.eigenvector_centrality_numpy(g)
    es = sorted_map(e)
    
    listeners, artists = bh.users()
    l = listeners['id'].tolist()
    a = artists['id'].tolist()
    l_list = [x for x in l if x in set(g.nodes())] #list of critics 
    a_list = [x for x in a if x in set(g.nodes())]
    
    if (es[-1][1]>0):
        es_critic = []
        es_artist = []
        
        for i in range(0,len(es)):
            if (es[i][0] in l_list):
                es_critic.append(es[i])
            else:
                es_artist.append(es[i])
                
        if (user_type == 'critic'):
            ans = es_critic[0:n]
        elif (user_type == 'artist'):
            ans = es_artist[0:n]
        else:
            ans =es[0:n]
        
        return(pd.DataFrame(ans, columns = ["id", "eigenvalue"]))
    else:
        return("No maximum eigenvalue found.")
        

In [ ]:
#Test function
n = 5
user_type = 'critic'
print("Top %d %ss by eigenvalue (%s, eigenvalue):" %(n,user_type, user_type))
getEigen(g_reduced, n, user_type)

In [ ]:
#Test function
n = 5
user_type = 'artist'
print("Top %d %ss by eigenvalue (%s, eigenvalue):" %(n,user_type, user_type))
getEigen(g_reduced, n, user_type)

In [ ]:
#Test function
n = 5
user_type = 'all user'
print("Top %d %ss by eigenvalue (%s, eigenvalue):" %(n,user_type, user_type))
getEigen(g_reduced, n, user_type)

<p><a name="centrality5"></a></p>
## Centrality Analysis #5: Page Rank

In [ ]:
#Function that returns top n "boundary spanners" by critic/artist

def getPgRank(g, n, user_type=None):
    p = nx.pagerank(g)
    ps=sorted_map(p)
    ps_critic = []
    ps_artist = []
    
    listeners, artists = bh.users()
    l = listeners['id'].tolist()
    a = artists['id'].tolist()
    l_list = [x for x in l if x in set(g.nodes())] #list of critics 
    a_list = [x for x in a if x in set(g.nodes())]
    
    for i in range(0,len(ps)):
        if (ps[i][0] in l_list):
            ps_critic.append(ps[i])
        else:
            ps_artist.append(ps[i])
    
    if (user_type == 'critic'):
        ans = ps_critic[0:n]
    elif (user_type == 'artist'):
        ans = ps_artist[0:n]
    else:
        ans = ps[0:n]
    return(pd.DataFrame(ans, columns = ["id", "page_rank"]))


In [ ]:
#Test function
n = 5
user_type = "critic"
print("Top %d %ss by PageRank (%s, page_rank):" %(n,user_type, user_type))
getPgRank(g_reduced, n, user_type)

In [ ]:
#Test function
n = 5
user_type = "artist"
print("Top %d %ss by PageRank (%s, page_rank):" %(n,user_type, user_type))
getPgRank(g_reduced, n, user_type)

In [ ]:
#Test function
n = 5
user_type = "all user"
print("Top %d %ss by PageRank (%s, page_rank):" %(n,user_type, user_type))
getPgRank(g_reduced, n, user_type)

<p><a name="centrality_summary"></a></p>
## Centrality Analysis: Summary

-- Define function that returns top users for all centrality measures

-- Define function that returns centrality measures for specific user

In [ ]:
#function that returns top n for each of the previous centrality measures in a data frame and sorts by
#number of connections
def topCentral(g, n, user_type):
    df_celeb = getCeleb(g, n, user_type)
    df_close = getClose(g, n, user_type)
    df_span = getSpan(g, n, user_type)
    df_eigen = getEigen(g, n, user_type)
    df_rank = getPgRank(g, n, user_type)
    
    names1 = df_celeb['id']
    names2 = df_close['id']
    names3 = df_span['id']
    
    if(type(df_eigen) == pd.DataFrame):
        names4 = df_eigen['id']
    else:
        names4 = names1
    names5 = df_rank['id']
    names=list(set(names1) | set(names2) | set (names3) | set (names4) | set (names5))
    
    df_celeb2 = getCeleb(g, len(g), user_type)
    df_close2 = getClose(g, len(g), user_type)
    df_span2 = getSpan(g, len(g), user_type)
    df_eigen2 = getEigen(g, len(g), user_type)
    df_rank2 = getPgRank(g, len(g), user_type)
    
    if(type(df_eigen) == pd.DataFrame):    
        table = [[name,df_celeb2.iloc[df_celeb2[df_celeb2['id']==name].index.values.astype(int)[0],1], \
            df_close2.iloc[df_close2[df_close2['id']==name].index.values.astype(int)[0],1], \
            df_span2.iloc[df_span2[df_span2['id']==name].index.values.astype(int)[0],1], \
            df_eigen2.iloc[df_eigen2[df_eigen2['id']==name].index.values.astype(int)[0],1],\
            df_rank2.iloc[df_rank2[df_rank2['id']==name].index.values.astype(int)[0],1]] for name in names] 
    
    else:
        table = [[name,df_celeb2.iloc[df_celeb2[df_celeb2['id']==name].index.values.astype(int)[0],1], \
            df_close2.iloc[df_close2[df_close2['id']==name].index.values.astype(int)[0],1], \
            df_span2.iloc[df_span2[df_span2['id']==name].index.values.astype(int)[0],1], \
            np.nan, df_rank2.iloc[df_rank2[df_rank2['id']==name].index.values.astype(int)[0],1]]
            for name in names] 
    df = pd.DataFrame(table[1:],columns= ['id',df_celeb.columns.get_values()[1], 
                      df_close.columns.get_values()[1], df_span.columns.get_values()[1],
                      "eigenvalue", df_rank.columns.get_values()[1]])
    
    df = df.sort_values(df_celeb.columns.get_values()[1], ascending = False)
    return(df)

In [ ]:
#Test getCentral function by iterating it through the various connection types

#Arguments passed to function
n = 3
user_type = 'critic'
connection = ['follow', 'message', 'view', 'sociallink']

for i in connection:
    df = get_dataframe(i)
    g = get_graph(df)
    g_reduced = filter_graph(g)
    print(i, " connection type", '\n')
    print("Summary of top %ss by various centrality measures, sorted by number of connections:" %( user_type),'\n')
    print(topCentral(g_reduced, n, user_type), '\n')

In [ ]:
#Define function that returns 1) data frame with the various centrality measures for a given user 
# and 2) second data frame with ranks 

def getCentral (user):
    connection = ['follow', 'message', 'view', 'sociallink']
    table1 =[] #table for centrality measures
    table2 =[] #table for centrality ranks
    
    for i in connection:
        temp_df = get_dataframe(i)
        g = get_graph(temp_df)
        g = filter_graph(g)
        
        #code for data frame with centrality measures
        d = nx.degree(g)[user]
        c = nx.closeness_centrality(g)[user]
        b = nx.betweenness_centrality(g)[user]
        p = nx.pagerank(g)[user]
        
        e = nx.eigenvector_centrality_numpy(g)
        es = sorted_map(e)
        
        if (es[-1][1]>0):
            e = nx.eigenvector_centrality_numpy(g)[user]
        else:
            e =np.nan
        
        table1.append([i,d, c, b, e, p])
        
        #code for data frame with centrality ranks
        max_ = len(g.nodes())
        df = getCeleb(g, max_)
        d_rank = df[df['id'] == user].index.values[0]+1
        df = getClose(g, max_)
        c_rank = df[df['id'] == user].index.values[0]+1
        df = getSpan(g, max_)
        b_rank = df[df['id'] == user].index.values[0]+1
        df = getPgRank(g, max_)
        p_rank = df[df['id'] == user].index.values[0]+1
        
        df = getEigen(g, max_)
        if (type(df) == pd.DataFrame):
            e_rank = df[df['id'] == user].index.values[0]+1
        else:
            e_rank =np.nan
            
        table2.append([i,d_rank, c_rank, b_rank, e_rank, p_rank])
     
    df1 = pd.DataFrame(table1, columns = ['connection_type', 'num_connections', 'closeness', 'betweenness', 
                                        'eigenvalue',  'page_rank'])
        
    df1.set_index('connection_type')
    
    df2 = pd.DataFrame(table2, columns = ['connection_type', 'num_connections', 'closeness', 'betweenness', 
                                        'eigenvalue',  'page_rank'])
        
    df2.set_index('connection_type')
    
        
    return df1, df2

In [ ]:
#Test getCentral function
getCentral(27)

<p><a name="ccc"></a></p>
## Components and Subgraphs

#### Subcomponents

In [ ]:
## define function that returns the top n components (graphs objects) by number of nodes
## this can possibly be fed into a pipeline for further analysis of the components
def getComponent(g, n):
    graphs = sorted(nx.connected_component_subgraphs(g), key = len, reverse=True)
    n_nodes = [nx.number_of_nodes(g) for g in graphs]
    
    if (n>len(graphs)):
        return("Number entered exceeds max value of %d. Please try again" %(len(graphs)))
    else:
        import heapq
        min_= min(heapq.nlargest(n, set(n_nodes)))        
        ans = [g for g in graphs if nx.number_of_nodes(g) >=min_]
        return(ans)

In [ ]:
##Test function
e=nx.read_pajek("egypt_retweets.net")
graphs = sorted(nx.connected_component_subgraphs(e),key = len, reverse=True)
n_nodes = [nx.number_of_nodes(g) for g in graphs]
getComponent(e, 5)

#### Islands

In [ ]:
#Define function that removes edges with weights below a certain level and returns
def trim_edges(g, weight=1): 
    g2=nx.Graph()
    for f, to, edata in g.edges(data=True): 
        if (edata['weight']) > weight:
            g2.add_edge(f,to,edata)
        return g2

In [ ]:
#Define a function to gradually trim the network into smaller networks. Returns threshold value and graph object.
#This can possibly be fed into a pipeline for further analysis of the islands.
def island_method(g, iterations=5):
    weights= [edata['weight'] for f,to,edata in g.edges(data=True)]
    mn=int(min(weights))
    mx=int(max(weights))
    step=int((mx-mn)/iterations)
    
    return [[threshold, trim_edges(g, threshold)] for threshold in range(mn,mx,step)]

In [ ]:
#test island_method function on biggest component
cc=getComponent(e,5)[0]
islands=island_method(cc)
islands

#### Ego graphs and networks

In [ ]:
#Define function that returns ego graph object for given user, connection type, depth of search

def getEgo (user, connection, depth):
    df = get_dataframe(connection)
    g = get_graph(df)
    g = filter_graph(g)
    ego = nx.Graph(nx.ego_graph(g,user, radius=depth))
    return ego

In [ ]:
#Test getEgo function
#Draw ego graph

user = getEgo(27, 'follow', 1)

color_map = {}
key_list = user.nodes()

for i in range(0, len(key_list)):
    if(user.node[key_list[i]]['user_type'] == 'critic'):
        color_map[key_list[i]] = 'b'
    else:
        color_map[key_list[i]] = 'r'

        
colors = [color_map.get(node) for node in user.nodes()]

plt.figure(figsize=(20,15))
nx.draw_networkx(user, node_color = colors, node_size = 500, alpha = 0.4)


In [ ]:
#Define function to get df for each user that shows number of connections and portion of connections that are
#connected to each other
def ego_graph_df(connection, user, degree=1):
    
    ego_graph = getEgo(user, connection, degree)
    num = len(ego_graph)
    avg = round(nx.average_clustering(ego_graph),2)
    ans = [[num, avg]]
    return (pd.DataFrame(ans, columns = ["num_connections", "percent_connected"]))

In [ ]:
ego_graph_df('follow',27,1)

### Triads

In [ ]:
#9/11 example from Book. Columns in the "edgelist.text" file, in order, are: 
#1)from, 2) to, 3) strength of tie, 4) level to which tie has been confirmed
#"attrib.txt" file contains the flight each person was on.

import csv 

with open('9_11_edgelist.txt', 'r') as f:
    reader = csv.reader(f) 
    g = nx.Graph()
    for row in reader:
        g.add_edge(row[0],row[1],weight=row[2],conf=row[3])
    
    for n in g.nodes_iter(): 
        g.node[n]['flight']='None'

with open('9_11_attrib.txt', 'r') as f:
    attribute = csv.reader(f) 
    for row in attribute:
        g.node[row[0]]['flight']=row[1]

##Examine largest subcomponent
components=sorted(nx.connected_component_subgraphs(g), key = len, reverse=True)
cc = components[0]

#Plot largest subcomponent. The function reads node attributes and assigns colors to 
#them based on attribute values. "multimode" function is defined in multimode.py file.

from collections import defaultdict
import multimode as mm

plt.figure(figsize=(20,15))
mm.plot_multimode(cc,type_string='flight')

In [ ]:
#Census of triad types using sample data. First number is number of bidirectional edges. 
#Second number is number of single edges. Third number is number of non-existent edges.
import triadic
import draw_triads
g=nx.DiGraph(nx.krackhardt_kite_graph())

census, node_census = triadic.triadic_census(g) 
census

In [ ]:
#Generate table of nodes with number of instances in each of the triad types

keys=node_census[0].keys()
## Generate a table header
print ('| Node |', ' | '.join(keys))
## Generate table contents
## A little magic is required to convert ints to strings 
for k in node_census.keys():
    print ('|', k, '|',' | '.join([str(v) for v in node_census[k].values()]))

In [ ]:
#See which users are in the most closed triads, i.e., cliques. First column is user, second is triads.
#Define function that returns count of closed triads each user is in by type of connection (follow, message)


def getCensus(connection, n, user_type = None):
    df = get_dataframe(i)
    g = get_graph(df)
    g = filter_graph(g)
    
    listeners, artists = bh.users()
    l = listeners['id'].tolist()
    a = artists['id'].tolist()
    l_list = [x for x in l if x in set(g.nodes())] #list of critics 
    a_list = [x for x in a if x in set(g.nodes())]
        
    census, node_census = triadic.triadic_census(g)
    
    if user_type =='critic':
        user_list = list(set(node_census.keys()) & set(l_list))
    elif user_type == 'artist':
        user_list = list(set(node_census.keys()) & set(a_list))
    else:
        user_list = node_census.keys()
    closed_triads=[[v,-k] for k,v in sorted([[-node_census[k]['300'],k] for k in user_list])]
    closed_triads_df = pd.DataFrame(closed_triads, columns = ['id', 'closed_triads'])
    return(closed_triads_df[0:n])

In [ ]:
#Test code
getCensus('follow', 5)

#### Hierarchical Clustering

In [ ]:
import hc

eco=nx.read_pajek("economic.net")
hc.create_hc(eco, 0)

## 2-Mode Networks

In [ ]:
#get dataframes for all four 2-mode networks
df_rating = get_dataframe('rating')
df_save = get_dataframe('save')
df_spotlight = get_dataframe('spotlight')
df_comment = get_dataframe('comment')

In [ ]:
df_rating

In [ ]:
# negate value of music_id column to make 2 non-intersecting sets 
# of node values: user_id, music_id
negate_musicid_1(df_rating)
negate_musicid_2(df_save)
negate_musicid_2(df_spotlight)
negate_musicid_1(df_comment)

In [ ]:
df_spotlight

In [ ]:
# Get bipartite graphs of each 2-mode connection type
BG_rating = get_bigraph_1(df_rating)
BG_save = get_bigraph_2(df_save)
BG_spotlight = get_bigraph_2(df_spotlight)
BG_comment = get_bigraph_1(df_comment)

In [ ]:
# Get lists for top and bottom nodes for each bipartite graph
rating_top, rating_bottom = get_bipartite_nodes(BG_rating)
save_top, save_bottom = get_bipartite_nodes(BG_save)
spotlight_top, spotlight_bottom = get_bipartite_nodes(BG_spotlight)
comment_top, comment_bottom = get_bipartite_nodes(BG_comment)

In [ ]:
# get positional info for how to plot nodes of bipartite graph
rating_pos = get_pos_1(df_rating)
save_pos = get_pos_2(df_save)
spotlight_pos = get_pos_2(df_spotlight)
comment_pos = get_pos_1(df_comment)

In [ ]:
plt.figure(figsize=(15,50))
nx.draw_networkx(BG_rating, rating_pos, node_size = 500, alpha = 0.3)

In [ ]:
plt.figure(figsize=(15,50))
nx.draw_networkx(BG_save, save_pos, node_size = 500, alpha = 0.3)

In [ ]:
plt.figure(figsize=(15,50))
nx.draw_networkx(BG_spotlight, spotlight_pos, node_size = 500, alpha = 0.3)

In [ ]:
plt.figure(figsize=(15,50))
nx.draw_networkx(BG_comment, comment_pos, node_size = 500, alpha = 0.3)

In [ ]:
WPG_rating_user = bipartite.weighted_projected_graph(BG_rating, rating_top)
WPG_rating_music = bipartite.weighted_projected_graph(BG_rating, rating_bottom)
WPG_save_user = bipartite.weighted_projected_graph(BG_rating, save_top)
WPG_save_music = bipartite.weighted_projected_graph(BG_rating, save_bottom)
WPG_spotlight_user = bipartite.weighted_projected_graph(BG_rating, spotlight_top)
WPG_spotlight_music = bipartite.weighted_projected_graph(BG_rating, spotlight_bottom)
WPG_comment_user = bipartite.weighted_projected_graph(BG_rating, comment_top)
WPG_comment_music = bipartite.weighted_projected_graph(BG_rating, comment_bottom)

In [ ]:
plt.figure(figsize=(15,50))
nx.draw_networkx(WPG_rating_user, node_size = 500, alpha = 0.3)

In [ ]:
plt.figure(figsize=(15,50))
nx.draw_networkx(WPG_rating_music, node_size = 500, alpha = 0.3)

In [ ]:
plt.figure(figsize=(15,50))
nx.draw_networkx(WPG_save_user, node_size = 500, alpha = 0.3)

In [ ]:
plt.figure(figsize=(15,50))
nx.draw_networkx(WPG_save_music, node_size = 500, alpha = 0.3)

In [ ]:
plt.figure(figsize=(15,50))
nx.draw_networkx(WPG_spotlight_user, node_size = 500, alpha = 0.3)

In [ ]:
plt.figure(figsize=(15,50))
nx.draw_networkx(WPG_spotlight_music, node_size = 500, alpha = 0.3)

In [ ]:
plt.figure(figsize=(15,50))
nx.draw_networkx(WPG_comment_user, node_size = 500, alpha = 0.3)

In [ ]:
plt.figure(figsize=(15,50))
nx.draw_networkx(WPG_comment_music, node_size = 500, alpha = 0.3)

In [ ]:
userlist_rating = get_userlists(df_rating)
musiclist_rating = get_musiclists1(df_rating)
userlist_comments = get_userlists(df_comments)
musiclist_comments = get_musiclists1(df_comments)
userlist_spotlighted = get_userlists(df_spotlighted)
musiclist_spotlighted = get_musiclists2(df_spotlighted)
userlist_saved = get_userlists(df_saved)
musiclist_saved = get_musiclists2(df_saved)

In [ ]:
music_rating_net = bi.weighted_projected_graph(GR, musiclist_rating, ratio = False)

In [ ]:
nx.is_connected(BG_rating)

In [ ]:
nx.is_bipartite(BG_rating)

In [ ]:
bottom_nodes, top_nodes = bipartite.sets(B)

In [ ]:
bottom_nodes, top_nodes = bipartite.sets(B)
top_nodes = {n for n, d in B.nodes(data=True) if d['bipartite']==0}
bottom_nodes = set(B) - top_nodes

In [ ]:
G=bipartite.projected_graph(B, top_nodes)

In [ ]:
bipartite.color(B)

In [ ]:
pos = {node:[0, i] for i,node in enumerate(df_rating['user_id'])}
pos.update({node:[1, i] for i,node in enumerate(df_rating['user_music_id'])})

In [ ]:
print(list(enumerate(df_rating['user_id'])))

In [ ]:
plt.figure(figsize=(15,50))
nx.draw_networkx(B, pos, node_size = 500, alpha = 0.3)

In [ ]:
B

In [ ]:
plt.figure(figsize=(20,15))
nx.draw_networkx(B, node_size = 500, alpha = 0.3)